## 1. 环境与依赖检查

如果你还没有按照 `requirements.txt` 安装依赖，可以使用下面的命令（可按需执行一次）：

```bash
pip install -r requirements.txt
```

在 Notebook 中也可以使用：

```python
# %pip install -r requirements.txt
```


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.chatanywhere.tech/v1")
LLM_MODEL = os.getenv("LLM_MODEL", "gpt-4o-mini")

print("📦 环境变量检查：")
print("  OPENAI_API_KEY:", "已配置" if OPENAI_API_KEY else "未配置 (将使用 DummyLLM)")
print("  SERPAPI_API_KEY:", "已配置" if SERPAPI_API_KEY else "未配置 (搜索工具将为占位实现)")
print("  OPENAI_BASE_URL:", OPENAI_BASE_URL)
print("  LLM_MODEL:", LLM_MODEL)

IS_DEEPSEEK = "deepseek.com" in (OPENAI_BASE_URL or "").lower()
if IS_DEEPSEEK:
    lm_lower = (LLM_MODEL or "").lower()
    if lm_lower not in ("deepseek-chat", "deepseek-reasoner"):
        new_model = "deepseek-reasoner" if ("reason" in lm_lower or "think" in lm_lower) else "deepseek-chat"
        print(f"ℹ️ 检测到 DeepSeek 接口，自动将模型名 '{LLM_MODEL}' 规范为 '{new_model}'。")
        LLM_MODEL = new_model


## 2. LLM 初始化（与脚本版本保持一致）

这里复用脚本中的三层回退策略：
1. 首选 `langchain_openai.ChatOpenAI`。
2. 失败时退到简易 HTTP 客户端 `HTTPFallbackChat`（直接调用 OpenAI 兼容接口）。
3. 再失败使用 `DummyLLM`，只输出占位文本，方便在无 Key 情况下调试流程。


In [ ]:
import json
import requests

class DummyLLM:
    """缺失 API Key 时使用的占位 LLM，实现 .invoke 接口。"""
    def __init__(self):
        self.model_name = "dummy-llm"
    def invoke(self, prompt):
        if isinstance(prompt, dict):
            return f"[DummyLLM response for keys: {list(prompt.keys())}]"
        return "[DummyLLM generic response]"
    def __or__(self, other):  # 兼容链式调用
        return other


class HTTPFallbackChat:
    """直接调用 OpenAI 兼容接口的轻量客户端，实现 .invoke(dict|str)。"""
    def __init__(self, base_url: str, api_key: str, model: str, timeout: float = 30.0):
        self.base_url = base_url.rstrip('/')
        self.api_key = api_key
        self.model = model
        self.timeout = timeout
        if self.base_url.endswith('/v1'):
            self.endpoint = f"{self.base_url}/chat/completions"
        else:
            self.endpoint = f"{self.base_url}/v1/chat/completions"

    def invoke(self, prompt):
        if isinstance(prompt, dict):
            user_content = '\n'.join(f"{k}: {v}" for k, v in prompt.items())
        else:
            user_content = str(prompt)
        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": "You are an academic writing optimization assistant."},
                {"role": "user", "content": user_content}
            ],
            "temperature": 0
        }
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        try:
            resp = requests.post(self.endpoint, headers=headers, json=payload, timeout=self.timeout)
            if resp.status_code != 200:
                return f"[HTTPFallbackChat Error {resp.status_code}: {resp.text[:200]}]"
            data = resp.json()
            return data.get("choices", [{}])[0].get("message", {}).get("content", "[No content]")
        except Exception as e:
            return f"[HTTPFallbackChat Exception: {e}]"

    def __or__(self, other):
        return other


def init_llm():
    if not OPENAI_API_KEY:
        print("⚠️ OPENAI_API_KEY 未配置，将使用 DummyLLM（仅用于流程调试）。")
        return DummyLLM()
    try:
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(
            model=LLM_MODEL,
            temperature=0,
            api_key=(lambda: OPENAI_API_KEY),
            base_url=OPENAI_BASE_URL,
        )
        print(f"✅ 主 LLM 已通过 ChatOpenAI 初始化: {LLM_MODEL} @ {OPENAI_BASE_URL}")
        return llm
    except Exception as e:
        print(f"❌ ChatOpenAI 初始化失败: {e}，尝试 HTTPFallbackChat...")
        try:
            fallback_llm = HTTPFallbackChat(OPENAI_BASE_URL, OPENAI_API_KEY, LLM_MODEL)
            probe = fallback_llm.invoke("probe")
            if probe.startswith("[HTTPFallbackChat Error") or probe.startswith("[HTTPFallbackChat Exception"):
                print("⚠️ HTTPFallbackChat 探测失败，回退到 DummyLLM。")
                return DummyLLM()
            print(f"✅ HTTPFallbackChat 已就绪: {LLM_MODEL} @ {OPENAI_BASE_URL}")
            return fallback_llm
        except Exception as e2:
            print(f"❌ HTTPFallbackChat 初始化失败: {e2}，回退到 DummyLLM。")
            return DummyLLM()


llm = init_llm()


## 3. 工具层：搜索 / Python REPL / 文件读写

这里与脚本版本保持一致：
- 如果没有 SerpAPI Key，则搜索工具返回占位提示，但接口仍然可用。
- 提供 Python REPL 工具执行简单代码片段。
- 提供最小的文本文件读写工具，方便在 Notebook 中快速试验。


In [4]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.utilities import SerpAPIWrapper

TOOLS = []

if SERPAPI_API_KEY:
    search_wrapper = SerpAPIWrapper(search_engine="google", serpapi_api_key=SERPAPI_API_KEY)
    search_tool = Tool(
        name="网络搜索",
        func=search_wrapper.run,
        description="实时信息查询：输入搜索关键词"
    )
    TOOLS.append(search_tool)
else:
    def _search_stub(q: str) -> str:
        return f"[SerpAPI 未配置，无法执行搜索: {q}]"
    search_tool = Tool(name="网络搜索", func=_search_stub, description="SerpAPI 未配置，占位搜索工具")
    TOOLS.append(search_tool)

python_repl = PythonREPL()
python_repl_tool = Tool(
    name="Python REPL",
    func=python_repl.run,
    description="执行格式正确的 Python 代码"
)

read_file_tool = Tool(
    name="读取文件",
    func=lambda fn: open(fn, "r", encoding="utf-8").read(),
    description="读取指定文本文件的全部内容，输入为文件路径"
)

write_file_tool = Tool(
    name="写入文件",
    func=lambda arg: (
        (lambda filename, content: (open(filename, "w", encoding="utf-8").write(content), "写入完成")[1])
    )(*arg.split(",", 1)),
    description="写入文件内容。输入格式: 文件名,内容（英文逗号分隔）"
)

TOOLS.extend([python_repl_tool, read_file_tool, write_file_tool])

print(f"🔧 已加载工具数量: {len(TOOLS)}")


已加载 4 个工具。


## 4. 向量记忆层：FAISS 或简易内存检索

与脚本版本保持一致：
- 优先使用 FAISS + OpenAIEmbeddings（或 DummyEmbeddings）。
- 如果 FAISS/依赖缺失，则退回到一个基于 token 重叠度的简易 `SimpleVectorStore`。


In [5]:
from typing import List, Dict, Optional, Tuple
import re
from datetime import datetime

EMBED_DIM = 1536
USE_FAISS = True
try:
    from langchain_community.vectorstores import FAISS
    from langchain_community.docstore.in_memory import InMemoryDocstore
    import faiss
    from langchain_core.documents import Document
    from langchain_core.embeddings import Embeddings as LCEmbeddings
except Exception as e:
    print(f"⚠️ 向量存储依赖缺失: {e}，使用简易内存检索替代 FAISS。")
    USE_FAISS = False
    FAISS = None  # type: ignore
    InMemoryDocstore = None  # type: ignore
    faiss = None  # type: ignore
    try:
        from langchain_core.documents import Document
    except Exception:
        class Document:  # 最小占位实现
            def __init__(self, page_content: str, metadata: Optional[Dict] = None):
                self.page_content = page_content
                self.metadata = metadata or {}

    class LCEmbeddings:  # type: ignore
        _dim = EMBED_DIM
        def embed_query(self, x: str):
            return [0.0] * self._dim
        def embed_documents(self, xs: List[str]):
            return [[0.0] * self._dim for _ in xs]


EMBED_MODEL_NAME = os.getenv("EMBED_MODEL_NAME", "text-embedding-3-small")

class DummyEmbeddings:
    def embed_query(self, t: str):
        return [0.0] * EMBED_DIM
    def embed_documents(self, docs: List[str]):
        return [[0.0] * EMBED_DIM for _ in docs]
    def __call__(self, t: str):
        return self.embed_query(t)


if OPENAI_API_KEY:
    try:
        if IS_DEEPSEEK:
            print("ℹ️ 检测到 DeepSeek base_url，跳过 OpenAIEmbeddings，使用占位 embedding。")
            embeddings_model = DummyEmbeddings()
        else:
            from langchain_openai import OpenAIEmbeddings
            embeddings_model = OpenAIEmbeddings(
                model=EMBED_MODEL_NAME,
                api_key=(lambda: OPENAI_API_KEY),
                base_url=OPENAI_BASE_URL
            )
            print(f"✅ Embeddings 模型已初始化: {EMBED_MODEL_NAME}")
    except Exception as e:
        print(f"❌ Embeddings 初始化失败: {e}，使用 DummyEmbeddings。")
        embeddings_model = DummyEmbeddings()
else:
    embeddings_model = DummyEmbeddings()
    print("⚠️ OPENAI_API_KEY 缺失，Embeddings 使用占位向量。")

_DEF_WORD_RE = re.compile(r"[\u4e00-\u9fff]|[A-Za-z0-9_]+")

def _simple_tokenize(text: str) -> List[str]:
    return _DEF_WORD_RE.findall(text or "")


if USE_FAISS:
    class EmbeddingAdapter(LCEmbeddings):
        def __init__(self, base):
            self.base = base
        def embed_query(self, x: str) -> List[float]:
            try:
                return self.base.embed_query(x)
            except Exception:
                return [0.0] * EMBED_DIM
        def embed_documents(self, xs: List[str]) -> List[List[float]]:
            try:
                return self.base.embed_documents(xs)
            except Exception:
                return [[0.0] * EMBED_DIM for _ in xs]

    adapter = EmbeddingAdapter(embeddings_model)
    index = faiss.IndexFlatL2(EMBED_DIM)
    vectorstore = FAISS(adapter, index, InMemoryDocstore({}), {})
    print("🧠 向量数据库(FAISS)已初始化。")
else:
    class SimpleVectorStore:
        def __init__(self):
            self.docs: List[Document] = []
        def add_documents(self, docs: List[Document]):
            self.docs.extend(docs)
        def similarity_search(self, query: str, k: int = 3) -> List[Document]:
            q_tokens = set(_simple_tokenize(query))
            def score(doc: Document):
                d_tokens = set(_simple_tokenize(doc.page_content))
                if not q_tokens or not d_tokens:
                    return 0.0
                return len(q_tokens & d_tokens) / len(q_tokens | d_tokens)
            ranked = sorted(self.docs, key=score, reverse=True)
            return ranked[:k]

    vectorstore = SimpleVectorStore()
    print("🧠 向量数据库简化版已初始化（无 FAISS）。")


class MemoryManager:
    def __init__(self, vs, namespace: str = "global"):
        self.vs = vs
        self.namespace = namespace
        self._counter = 0
    def add_memory(self, text: str, metadata: Optional[Dict] = None):
        try:
            meta = metadata or {}
            meta.update({"namespace": self.namespace, "ts": datetime.now().isoformat()})
            doc = Document(page_content=text, metadata=meta)
            self.vs.add_documents([doc])
            self._counter += 1
        except Exception as e:
            print(f"⚠️ 写入记忆失败: {e}")
    def recall(self, query: str, k: int = 3) -> List[str]:
        try:
            res = self.vs.similarity_search(query, k=k)
            return [d.page_content for d in res]
        except Exception as e:
            print(f"⚠️ 读取记忆失败: {e}")
            return []


向量数据库记忆模块已初始化。



## 5. 双 Agent 学术表达优化系统（与脚本版本同步）

这里直接复用 `multi_agent_nlp_project.py` 中的实现，只保留与“协作优化”强相关的部分，方便在 Notebook 中交互式调用。


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import difflib
import time


class DualAgentAcademicSystem:
    def __init__(self, llm, tools, vectorstore, enable_tools: bool = True, enable_memory: bool = True):
        self.llm = llm
        self.tools_enabled = enable_tools
        self.memory_enabled = enable_memory
        self.tools = {t.name: t for t in tools}
        self.vectorstore = vectorstore
        self.memory = MemoryManager(vectorstore) if enable_memory else None
        self.collaboration_log: List[Dict] = []
        self._setup_agents()

    def _setup_agents(self):
        self.agent_a_template = PromptTemplate.from_template(
            """
你是Agent A - 学术表达优化专家。
轮次: 第{round_num}轮
用户需求: {user_requirements}
上一轮评分(若有): {last_scores}
长程记忆检索片段:
{memory_snippets}
工具观察:
{tool_observations}
待优化文本:
{text_to_optimize}
{previous_feedback}

请输出：
**优化版本：**
[优化后的完整文本]

**修改说明：**
[说明本轮修改要点，尤其针对评审提出的高优先级问题]
"""
        )

        self.agent_b_template = PromptTemplate.from_template(
            """
你是Agent B - 学术评审与对抗质询专家。
轮次: 第{round_num}轮
用户需求: {user_requirements}
优化文本:
{optimized_text}

请评审并输出(严格包含以下板块与数值)：
**本轮改进评价：**
[总体评价]

**评分(请使用JSON格式)**
{{"quality": <1-10>, "rigor": <1-10>, "logic": <1-10>, "novelty": <1-10>, "priority_issues": <描述>}}

**剩余主要问题：**
[...]

**下轮重点建议：**
1. [...]
2. [...]

**改进优先级：**
[高/中/低 分层列出]
"""
        )

        self.agent_a_chain = self.agent_a_template | self.llm | StrOutputParser()
        self.agent_b_chain = self.agent_b_template | self.llm | StrOutputParser()

    @staticmethod
    def _extract_section(text: str, start_token: str, end_token: str) -> str:
        lines = text.split('\n')
        collecting = False
        buf = []
        for l in lines:
            if start_token in l:
                collecting = True
                continue
            if collecting and end_token in l:
                break
            if collecting:
                buf.append(l)
        return '\n'.join(buf).strip()

    def _compute_diff(self, prev: str, current: str) -> str:
        if prev is None:
            return '(首轮无diff)'
        diff_lines = difflib.unified_diff(prev.splitlines(), current.splitlines(), lineterm='')
        collected = []
        for i, line in enumerate(diff_lines):
            if i > 400:
                collected.append('... <diff truncated>')
                break
            collected.append(line)
        return '\n'.join(collected) if collected else '(无变化)'

    def _parse_scores(self, feedback: str) -> Dict[str, float]:
        import json as _json, re as _re
        m = _re.search(r'\{\s*"quality".*?\}', feedback, flags=_re.S)
        if not m:
            return {}
        blob = m.group(0)
        try:
            data = _json.loads(blob)
            for k in ["quality", "rigor", "logic", "novelty"]:
                if k in data:
                    data[k] = float(data[k])
            return data
        except Exception:
            return {}

    def _plan_and_act(self, text: str, requirements: List[str]) -> str:
        if not self.tools_enabled:
            return "(工具已禁用)"
        observations = []
        joined_req = ' '.join(requirements).lower()
        if any(kw in joined_req for kw in ["search", "检索", "事实", "最新", "引用"]):
            m = re.findall(r'"([^\"]+)"', text)
            query = m[-1] if m else text.split('。')[-1].strip() or text
            try:
                obs = self.tools["网络搜索"].run(query)
            except Exception as e:
                obs = f"[搜索异常: {e}]"
            observations.append(f"搜索[{query}] -> {obs[:300]}")
        return "\n".join(observations) if observations else "(无)"

    def collaborate(self, user_text: str, user_requirements: List[str], language: str = "中文", rounds: int = 3) -> Tuple[str, List[Dict]]:
        self.collaboration_log = [{"round": 0, "user_input": user_text, "requirements": user_requirements, "timestamp": datetime.now().isoformat()}]
        current_text = user_text
        previous_feedback = ""
        last_scores: Dict[str, float] = {}

        if self.memory_enabled:
            self.memory.add_memory(user_text, {"type": "user_input"})

        for r in range(1, rounds + 1):
            mem_snippets = []
            if self.memory_enabled:
                mem_snippets = self.memory.recall(current_text, k=3)
            tool_obs = self._plan_and_act(current_text, user_requirements)

            a_input = {
                "round_num": r,
                "text_to_optimize": current_text,
                "user_requirements": ', '.join(user_requirements),
                "previous_feedback": previous_feedback,
                "memory_snippets": '\n'.join(mem_snippets) if mem_snippets else "(无)",
                "tool_observations": tool_obs,
                "last_scores": last_scores if last_scores else "(无)"
            }
            a_resp = self.agent_a_chain.invoke(a_input)
            optimized_text = self._extract_section(a_resp, "**优化版本：**", "**修改说明：**") or current_text

            b_input = {
                "round_num": r,
                "optimized_text": optimized_text,
                "user_requirements": ', '.join(user_requirements)
            }
            b_resp = self.agent_b_chain.invoke(b_input)

            last_scores = self._parse_scores(b_resp)
            diff_str = self._compute_diff(current_text, optimized_text)

            if self.memory_enabled:
                self.memory.add_memory(optimized_text, {"type": "optimized_text", "round": r})
                self.memory.add_memory(b_resp, {"type": "feedback", "round": r})

            self.collaboration_log.append({
                "round": r,
                "agent_a_response": a_resp,
                "optimized_text": optimized_text,
                "agent_b_feedback": b_resp,
                "scores": last_scores,
                "tool_observations": tool_obs,
                "diff": diff_str,
                "timestamp": datetime.now().isoformat()
            })

            previous_feedback = b_resp
            current_text = optimized_text
            print(f"✅ Round {r} 完成 | 评分: {last_scores if last_scores else '{}'}")
            time.sleep(0.15)

        return current_text, self.collaboration_log


dual_agent_system = DualAgentAcademicSystem(llm, TOOLS, vectorstore)
print("🤖 双Agent系统初始化完成（Notebook 版）")


## 6. Notebook 中的使用示例

下面给出几个典型用法示例：
- 单段文本多轮学术优化。
- 调整需求维度、轮次数。
- 观察多轮协作日志结构。


In [ ]:
from pprint import pprint

sample_text = "这是一段关于多智能体协作进行学术写作优化的初稿，表述略显口语化，结构也不够清晰。"
requirements = ["学术表达提升", "逻辑结构优化"]

final_text, log = dual_agent_system.collaborate(sample_text, requirements, rounds=2)

print("\n📌 最终优化文本:\n")
print(final_text)

print("\n📜 协作日志摘要（最后一轮）:\n")
pprint(log[-1])


> 你可以在本 Notebook 中根据需要继续添加：
> - 数据合成 / 蒸馏对准备的实验性单元格，直接调用 `multi_agent_nlp_project.py` 中的 CLI；
> - 将长文本按段拆分后调用 `dual_agent_system.collaborate` 的 Notebook 版本封装；
> - 或者简单作为脚本行为的可视化与调试环境。

